<p><font size="6" color='grey'> <b>
Anwendung Generativer KI
</b></font> </br></p>

<p><font size="5" color='grey'> <b>
Multimodal - Bild
</b></font> </br></p>


---

In [ ]:
#@title
#@markdown   <p><font size="4" color='green'>  Colab-Umfeld</font> </br></p>
# Installierte Python Version
import sys
print(f"Python Version: ",sys.version)

# Installierte LangChain Bibliotheken
print()
print("Installierte LangChain Bibliotheken:")
!pip list | grep '^langchain'
# Unterdrückt die "DeprecationWarning" von LangChain für die Memory-Funktionden
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning, module="langsmith.client")

In [ ]:
#@title
#@markdown   <p><font size="4" color='green'>  SetUp API-Keys (setup_api_keys)</font> </br></p>

def setup_api_keys():
    """Konfiguriert alle benötigten API-Keys aus Google Colab userdata"""
    from google.colab import userdata
    import os
    from os import environ

    # Dictionary der benötigten API-Keys
    keys = {
        'OPENAI_API_KEY': 'OPENAI_API_KEY',
        'HF_TOKEN': 'HF_TOKEN',
        # Weitere Keys bei Bedarf
    }

    # Keys in Umgebungsvariablen setzen
    for env_var, key_name in keys.items():
        environ[env_var] = userdata.get(key_name)

    return {k: environ[k] for k in keys.keys()}

# Verwendung
all_keys = setup_api_keys()
# Bei Bedarf einzelne Keys direkt zugreifen
# WEATHER_API_KEY = all_keys['WEATHER_API_KEY']



# 1 | Einführung
---

Dieses Modul beschäftigt sich mit der Analyse von Bildern und ihrer Rolle als Basis für multimodale Systeme. Solche Modelle stellen einen bedeutenden Fortschritt in der KI dar, da sie geschriebene Sprache in visuelle Darstellungen umwandeln. Bekannte Beispiele wie DALL·E, Midjourney und Stable Diffusion nutzen Deep-Learning-Techniken, um detaillierte Bilder auf Grundlage von Textbeschreibungen zu erzeugen. Dabei erfassen sie die Zusammenhänge zwischen sprachlichen und visuellen Elementen, wodurch sie in der Lage sind, Szenen, Objekte und Konzepte entsprechend der gegebenen Beschreibung zu generieren.  

Multimodale Modelle gehen über diese Funktionalität hinaus, indem sie mehrere Datenquellen kombinieren – in der Regel Text,  Bilder, Audio oder Video. Ihr Ziel ist es, verschiedene Informationsarten simultan zu verarbeiten und in Beziehung zu setzen. So können sie beispielsweise Bildbeschreibungen aus Text erstellen, Fragen zu visuellen Inhalten beantworten oder sogar Videos auf Basis von Texteingaben generieren. Durch die Verbindung der Fähigkeiten von Text- und Bildmodellen erweitern multimodale Systeme das Anwendungsspektrum der KI und ermöglichen eine flexiblere, kontextbezogene Interaktion.  

Während **Text-zu-Bild-Modelle** primär darauf ausgerichtet sind, sprachliche Eingaben in visuelle Inhalte umzusetzen, ermöglichen **multimodale Modelle** eine komplexere Verknüpfung verschiedener Medientypen. In diesem Modul wird untersucht, wie diese Modelle funktionieren und wie ihre Architektur die Entwicklung weiterführender multimodaler Systeme beeinflusst. Zudem werden praxisnahe Anwendungsfälle betrachtet sowie mögliche zukünftige Entwicklungen aufgezeigt, die sich aus der Kombination unterschiedlicher Datenmodalitäten ergeben.


# 2 | Bildgenerierung
---

DALL·E ist ein von OpenAI entwickeltes generatives KI-Modell, das aus Textbeschreibungen Bilder erzeugt. Es kann detaillierte Anweisungen interpretieren und eine breite Palette von Motiven und Stilen – von realistisch bis abstrakt – visuell umsetzen.

Über die OpenAI-API ist der Zugriff auf DALL·E möglich, wodurch Entwickler die Bildgenerierungsfunktion in ihre Anwendungen integrieren können. Nutzer geben Texteingaben vor, anhand derer das Modell passende Bilder erstellt. Die API ist so konzipiert, dass sie sich einfach einbinden und skalieren lässt, wodurch sie sich für Webanwendungen, mobile Apps und digitale Plattformen eignet.

Mit DALL·E können Ideen in hochdetaillierte visuelle Darstellungen umgewandelt werden. Das Modell findet in verschiedenen Bereichen Anwendung, darunter Design, Marketing, Unterhaltung und Bildung.


<p><font color='black' size="5">
DALL·E 3
</font></p>

DALL·E 3, veröffentlicht im Jahr 2023 und bringt neue Funktionen mit, die die Präzision und Benutzerfreundlichkeit verbessern:

* **Optimiertes Textverständnis**: DALL·E 3 kann detaillierte und komplexe Anweisungen interpretieren und umsetzen. Dadurch werden Bilder erzeugt, die den Vorstellungen des Nutzers besser entsprechen, selbst bei anspruchsvollen oder feingliedrigen Beschreibungen.  
* **Nahtlose Integration mit ChatGPT**: Dank der tiefen Verzahnung mit ChatGPT können Nutzer in einem dialogbasierten Prozess ihre Eingaben iterativ verfeinern. Dies erleichtert die Anpassung von Anweisungen und macht die Bildgenerierung intuitiver.  
* **Verbesserte Darstellung schwieriger Elemente**: Herausforderungen früherer Versionen, etwa die realistische Darstellung von Händen und Gesichtern, wurden erheblich reduziert, sodass DALL·E 3 in diesen Bereichen präzisere Ergebnisse liefert.  

<p><font color='black' size="5">
Mehrere Bilder generieren
</font></p>

Mit DALL·E 3 lassen sich mehrere Bilder erzeugen, die in diesem Abschnitt betrachtet werden.  

Die Auflösung kann mithilfe der OpenAI-Bibliothek festgelegt werden, wobei nicht jede Auflösung für alle Modelle verfügbar ist:  

* 1024 × 512  
* 256 × 256  
* 512 × 512  
* 1024 × 1024  
* 1024 × 1792  
* 1792 × 1024  
* Qualitätsstufen: Standard/hd  

<p><font color='black' size="5">
Erläuterung des Codes
</font></p>


Der folgende Code erzeugt das Bild eines "schwarzen Pferdes“.

In [ ]:
import requests
from openai import OpenAI
from io import BytesIO
from PIL import Image
from IPython.display import display

# Parameter
MODEL = "dall-e-3"
PROMPT = "Erstelle ein Bild mit einer Hütte im Wald, die Hütte wird von der Sonne beschienen."
NUM_IMAGES = 2
DISPLAY_SIZE = (512, 512)  # Anzeigegröße der Bilder (Breite, Höhe)

def generate_and_save_images():
    client = OpenAI()

    for i in range(NUM_IMAGES):
        response = client.images.generate(model=MODEL, prompt=PROMPT, size="1024x1024", quality="standard", n=1)
        img = Image.open(BytesIO(requests.get(response.data[0].url).content))

        # Bild speichern
        filename = f"bild_{i+1}.png"
        img.save(filename)
        print(f"Gespeichert: {filename}")

        # Bildgröße anpassen und anzeigen
        img_resized = img.resize(DISPLAY_SIZE)
        display(img_resized)


generate_and_save_images()

<p><font color='black' size="5">
Bild herunterladen
</font></p>

In [ ]:
from google.colab import files
# Laden Sie das als variation_image_1.png gespeicherte Bild herunter
files.download('/content/bild_2.png')


# 3 | Bildklassifizierung
---

Die Bildklassifizierung ist eine grundlegende Aufgabe in der Computer Vision, bei der einem gesamten Bild eine einzige Kategorie (Klasse) zugewiesen wird.

**Wie funktioniert Bildklassifizierung?**
+ Eingabe: Ein einzelnes Bild wird als Eingabe verwendet.
+ Merkmalsextraktion: Ein neuronales Netzwerk analysiert das Bild und extrahiert relevante Merkmale (z. B. Kanten, Farben, Formen, Muster).
+ Klassifizierung: Das Modell ordnet das Bild einer vordefinierten Kategorie zu, z. B. "Hund", "Katze" oder "Auto".
+ Ausgabe: Eine einzige Klasse (Label) mit einer Wahrscheinlichkeitsbewertung wird zurückgegeben.


Beispiel für Bildklassifizierung
Stelle dir vor, du hast ein Bild von einem Hund. Ein Bildklassifizierungsmodell verarbeitet das Bild und gibt die Kategorie "Hund" mit einer bestimmten Wahrscheinlichkeit (z. B. 95 %) zurück.

+ Eingabebild: Ein Bild eines Hundes
+ Modell-Ausgabe: "Hund" (95%)

Falls das Bild eine Katze zeigt, gibt das Modell möglicherweise "Katze" (90%) als Ergebnis zurück.

**Einschränkungen der Bildklassifizierung**     
Das Modell kann nur eine Klasse pro Bild erkennen, auch wenn mehrere Objekte im Bild vorhanden sind.
Es gibt keine Information über die Position oder Anzahl der Objekte im Bild.
Anwendungsfälle für Bildklassifizierung
+ Erkennung von medizinischen Anomalien (z. B. Klassifikation von Röntgenbildern)
+ Identifikation von Pflanzen oder Tieren anhand von Bildern
+ Sentiment-Analyse anhand von Gesichtsmerkmalen

**Bekannte Modelle für Bildklassifizierung**    
+ CNNs (Convolutional Neural Networks) wie ResNet, VGG, EfficientNet
+ Pretrained Modelle: MobileNet, Inception, AlexNet


Man kann allerdings auch gpt-4o-mini mit einem entsprechenden Prompt hierzu einsetzen.

<p><font color='black' size="5">
Erläuterung des Codes
</font></p>

Text

In [ ]:
!uv pip install --system --prerelease allow openai pillow

In [ ]:
import base64
from openai import OpenAI
from google.colab import files
from IPython.display import Image, display, Markdown

client = OpenAI()

# Datei-Auswahldialog öffnen
uploaded = files.upload()

# Dateinamen abrufen (das erste hochgeladene Bild)
image_path = list(uploaded.keys())[0]

# Bild in Base64 kodieren
encoded_image = base64.b64encode(uploaded[image_path]).decode("utf-8")

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "user",
            "content": [
                {"type": "text", "text": "Was ist auf diesem Bild abgebildet? Nennen nur das wesentliche Objekt."},
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/jpeg;base64,{encoded_image}"
                    },
                },
            ],
        }
    ],
    max_tokens=300,
)

print(response.choices[0].message.content)

# Bild in Colab anzeigen mit begrenzter Größe
display(Image(uploaded[image_path], width=500))  # Höhe wird proportional angepasst


# 4 | Objekterkennung
---

Die Objekterkennung geht einen Schritt weiter als die Bildklassifizierung. Hier wird nicht nur bestimmt, welche Objekte in einem Bild vorhanden sind, sondern auch wo sie sich befinden.

**Wie funktioniert Objekterkennung?**   
+ Eingabe: Ein einzelnes Bild wird als Eingabe verwendet.
+ Merkmalsextraktion & Vorschläge für Regionen: Das Modell analysiert das Bild und identifiziert potenzielle Bereiche, in denen sich Objekte befinden könnten.
+ Klassifizierung & Lokalisierung: Jedes erkannte Objekt wird einer bestimmten Klasse zugeordnet, und die Position wird durch eine Bounding Box (ein rechteckiger Bereich um das Objekt) beschrieben.
+ Ausgabe: Eine Liste mit allen erkannten Objekten, ihrer Klasse und ihrer Position im Bild wird zurückgegeben.


**Beispiel für Objekterkennung**    
Stelle dir vor, du hast ein Bild mit einem Hund, einer Katze und einem Auto. Ein Objekterkennungsmodell kann alle drei Objekte gleichzeitig identifizieren und ihre Positionen angeben.

+ Eingabebild: Ein Bild mit einem Hund, einer Katze und einem Auto
+ Modell-Ausgabe:
>Hund (95%) - Position: (x1, y1, x2, y2)   
Katze (90%) - Position: (x3, y3, x4, y4)   
Auto (99%) - Position: (x5, y5, x6, y6)   

Hierbei sind (x1, y1, x2, y2) die Koordinaten der Bounding Box für den Hund, (x3, y3, x4, y4) für die Katze usw.

**Einschränkungen der Objekterkennung**   
Aufwendigere Berechnungen im Vergleich zur einfachen Bildklassifizierung.
Schwieriger zu trainieren, da genaue Positionen für Trainingsdaten erforderlich sind.
Anwendungsfälle für Objekterkennung
+ Autonomes Fahren (Erkennung von Fußgängern, Verkehrsschildern)
+ Überwachungssysteme (Erkennen von Eindringlingen oder gefährlichen Objekten)
+ Analyse von Bildern im Einzelhandel (z. B. automatische Produktzählung in Regalen)

**Bekannte Modelle für Objekterkennung**   
+ YOLO (You Only Look Once) – sehr schnell und effizient
+ Faster R-CNN – präzise, aber langsamer
+ SSD (Single Shot MultiBox Detector) – guter Kompromiss zwischen Geschwindigkeit und Genauigkeit

<p><font color='black' size="5">
YOLO
</font></p>

**YOLO (You Only Look Once)** ist ein leistungsfähiger Algorithmus für die Objekterkennung in Bildern und Videos mithilfe künstlicher Intelligenz (KI). Es handelt sich um ein Modell, das auf Convolutional Neural Networks (CNN) basiert und sich durch seine Geschwindigkeit und Genauigkeit auszeichnet.

**Funktionsweise**

YOLO teilt ein Bild in ein Gitter auf und verarbeitet es in einem einzigen Durchlauf. Jede Gitterzelle ist für die Erkennung von Objekten in ihrem Bereich verantwortlich. Der Algorithmus sagt gleichzeitig Begrenzungsrahmen (Bounding Boxes) und Klassenwahrscheinlichkeiten für Objekte voraus.

**Vorteile**

1. **Geschwindigkeit**: YOLO kann Bilder in Echtzeit mit bis zu 45 Bildern pro Sekunde verarbeiten, was es ideal für Anwendungen wie Videoüberwachung oder autonomes Fahren macht.

2. **Genauigkeit**: Trotz seiner Geschwindigkeit erreicht YOLO eine hohe Erkennungsgenauigkeit und weist nur wenige Hintergrundfehler auf.

3. **Verallgemeinerungsfähigkeit**: Neuere Versionen von YOLO bieten eine verbesserte Leistung bei der Erkennung von Objekten in neuen Umgebungen.

**Entwicklung**

Seit seiner Einführung im Jahr 2015 hat YOLO mehrere Iterationen durchlaufen, wobei jede Version Verbesserungen und Optimierungen mit sich brachte. Die neueste Version ist YOLO v7, die weitere Fortschritte in Bezug auf Geschwindigkeit und Genauigkeit bietet.

YOLO hat die Objekterkennung revolutioniert und findet Anwendung in verschiedenen Bereichen wie autonomen Fahrzeugen, Überwachungssystemen, Robotik und generativer KI.

[Ultralytics](https://www.ultralytics.com/de)

In [ ]:
!uv pip install --system --prerelease allow ultralytics opencv-python

<p><font color='black' size="5">
Erläuterung des Codes
</font></p>

Der Code verwendet die Ultralytics YOLOv8 Bibliothek, um Objekterkennung auf einem hochgeladenen Bild durchzuführen. Er lädt ein vortrainiertes YOLOv8m-Modell, ermöglicht das Hochladen eines Bildes über Google Colab, führt die Objekterkennung auf diesem Bild durch (mit einer Konfidenzschwelle von 0.2), speichert das Bild mit den Erkennungsergebnissen und die entsprechenden Textdateien und zeigt schließlich das bearbeitete Bild in der Colab-Ausgabe an.

In [ ]:
# Import
import ultralytics
from ultralytics import YOLO
from google.colab import files
from IPython.display import Image, display, Markdown

# YOLO-Modell laden
model = YOLO("yolov8m.pt")

# Datei-Auswahldialog öffnen
uploaded = files.upload()

# Dateinamen abrufen (das erste hochgeladene Bild)
image_path = list(uploaded.keys())[0]

# YOLO-Vorhersage durchführen
results = model.predict(source=image_path, save=True, conf=0.2, save_txt=True)

# Pfad zum gespeicherten Bild mit den Erkennungen
result_image_path = results[0].save_dir + "/" + image_path

# Bild mit erkannten Objekten anzeigen
display(Image(result_image_path,width=600)) # Höhe wird automatisch angepasst

# 5 | Bildbeschreibung

Die Aufgabe Bildbeschreibung (Image2Text) beschreibt die automatische Generierung von Text aus Bildern mithilfe von Künstlicher Intelligenz (KI). Dabei werden neuronale Netze, insbesondere tiefgehende Modelle in Kombination mit Transformern, genutzt, um visuelle Inhalte zu analysieren und zu beschreiben.

**Anwendungsfälle**    
+ Bildbeschreibung (Image Captioning): Generierung von Textbeschreibungen zu Bildern (z. B. für barrierefreie Webseiten oder Archivierung).
+ Optische Zeichenerkennung (OCR): Extraktion von Text aus Bildern oder gescannten Dokumenten.
+ Visuelle Frage-Antwort-Systeme (Visual Question Answering, VQA): Beantwortung von Fragen zu Bildinhalten.
+ Content-Moderation: Automatische Identifikation und Beschreibung von problematischen oder sensiblen Bildinhalten.

In [ ]:
!uv pip install --system --prerelease allow openai pillow

<p><font color='black' size="5">
Erläuterung des Codes
</font></p>

Der Code lädt ein Bild in Google Colab hoch, kodiert es in Base64 und sendet es zusammen mit einer Frage ("Was ist auf dem Bild?") an das OpenAI-Modell "gpt-4o-mini". Die Antwort des Modells (eine detaillierte Beschreibung im Markdown-Format) wird angezeigt, ebenso wie das hochgeladene Bild selbst (mit einer maximalen Breite von 500 Pixeln).


In [ ]:
import base64
from openai import OpenAI
from google.colab import files
from IPython.display import Image, display, Markdown

client = OpenAI()

# Datei-Auswahldialog öffnen
uploaded = files.upload()

# Dateinamen abrufen (das erste hochgeladene Bild)
image_path = list(uploaded.keys())[0]

# Bild in Base64 kodieren
encoded_image = base64.b64encode(uploaded[image_path]).decode("utf-8")

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "user",
            "content": [
                {"type": "text", "text": "Wss ist auf dem Bild? Erstelle eine ausführliche Beschreibung im Markdown-Format."},
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/jpeg;base64,{encoded_image}"  # Oder image/png, etc.
                    },
                },
            ],
        }
    ],
    max_tokens=300,
)

# Bildbeschreibung
display(Markdown(response.choices[0].message.content))

# Bild in Colab anzeigen mit begrenzter Größe
display(Image(uploaded[image_path], width=500))  # Höhe wird proportional angepasst


# A | Aufgabe
---

Die Aufgabestellungen unten bieten Anregungen, Sie können aber auch gerne eine andere Herausforderung angehen.

<p><font color='black' size="5">
Coverbild LLM-Buch
</font></p>

In einem früheren Modul wurde ein LLM-basierten Buchgenerator erstellt, der die Erstellung eines Buches von Anfang bis Ende automatisiert. Dieser Prozess umfasste die Erstellung einer Zusammenfassung, die Gliederung des Inhaltsverzeichnisses und das iterative Schreiben der Kapitel in ein strukturiertes Markdown-Dokument. Jetzt wird diese Arbeit erweitert: Erstellen Sie mit DALL·E ein Coverbild für das Buch, das den textbasierten Inhalt ergänzt.

